In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import BertTokenizer


In [ ]:
df = pd.read_csv("hf://datasets/merve/poetry/poetry.csv")

In [ ]:
df

,author,content,poem name,age,type
0,WILLIAM SHAKESPEARE,Let the bird of loudest lay\r\nOn the sole Ara...,The Phoenix and the Turtle,Renaissance,Mythology & Folklore
1,DUCHESS OF NEWCASTLE MARGARET CAVENDISH,"Sir Charles into my chamber coming in,\r\nWhen...",An Epilogue to the Above,Renaissance,Mythology & Folklore
2,THOMAS BASTARD,"Our vice runs beyond all that old men saw,\r\n...","Book 7, Epigram 42",Renaissance,Mythology & Folklore
3,EDMUND SPENSER,"Lo I the man, whose Muse whilome did maske,\r\...","from The Faerie Queene: Book I, Canto I",Renaissance,Mythology & Folklore
4,RICHARD BARNFIELD,"Long have I longd to see my love againe,\r\nSt...",Sonnet 16,Renaissance,Mythology & Folklore
...,...,...,...,...,...
568,SARA TEASDALE,"With the man I love who loves me not,\r\nI wal...",Union Square,Modern,Love
569,HART CRANE,"Hart Crane, ""Voyages I, II, III, IV, V, VI"" fr...",Voyages,Modern,Love
570,WILLIAM BUTLER YEATS,"When you are old and grey and full of sleep,\r...",When You Are Old,Modern,Love
571,CARL SANDBURG,"Give me hunger,\r\nO you gods that sit and giv...",At a Window,Modern,Love


In [ ]:
df['training set'] = '<|theme|> ' + df['type'] + ' <|title|> ' + df['poem name'] + ' <|poem|> ' + df['content']
df

,author,content,poem name,age,type,training set
0,WILLIAM SHAKESPEARE,Let the bird of loudest lay\r\nOn the sole Ara...,The Phoenix and the Turtle,Renaissance,Mythology & Folklore,<|theme|> Mythology & Folklore <|title|> The P...
1,DUCHESS OF NEWCASTLE MARGARET CAVENDISH,"Sir Charles into my chamber coming in,\r\nWhen...",An Epilogue to the Above,Renaissance,Mythology & Folklore,<|theme|> Mythology & Folklore <|title|> An Ep...
2,THOMAS BASTARD,"Our vice runs beyond all that old men saw,\r\n...","Book 7, Epigram 42",Renaissance,Mythology & Folklore,<|theme|> Mythology & Folklore <|title|> Book ...
3,EDMUND SPENSER,"Lo I the man, whose Muse whilome did maske,\r\...","from The Faerie Queene: Book I, Canto I",Renaissance,Mythology & Folklore,<|theme|> Mythology & Folklore <|title|> from ...
4,RICHARD BARNFIELD,"Long have I longd to see my love againe,\r\nSt...",Sonnet 16,Renaissance,Mythology & Folklore,<|theme|> Mythology & Folklore <|title|> Sonne...
...,...,...,...,...,...,...
568,SARA TEASDALE,"With the man I love who loves me not,\r\nI wal...",Union Square,Modern,Love,<|theme|> Love <|title|> Union Square <|poem|>...
569,HART CRANE,"Hart Crane, ""Voyages I, II, III, IV, V, VI"" fr...",Voyages,Modern,Love,<|theme|> Love <|title|> Voyages <|poem|> Hart...
570,WILLIAM BUTLER YEATS,"When you are old and grey and full of sleep,\r...",When You Are Old,Modern,Love,<|theme|> Love <|title|> When You Are Old <|po...
571,CARL SANDBURG,"Give me hunger,\r\nO you gods that sit and giv...",At a Window,Modern,Love,<|theme|> Love <|title|> At a Window <|poem|> ...


In [ ]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [ ]:
special_tokens = {
    "additional_special_tokens": ["<|theme|>", "<|title|>" ,"<|poem|>"]
}
tokenizer.add_special_tokens(special_tokens)

# After adding special tokens (if you did)
vocab_size = len(tokenizer)  # ✅ Correct vocab size

In [ ]:
texts = df['training set'].fillna("").astype(str).tolist()

# Tokenize (with padding + truncation)
encoding = tokenizer(
    texts,
    padding=True,
    truncation=True,
    return_tensors='pt'  # this ensures output is a PyTorch tensor
)

In [ ]:
input_ids = encoding['input_ids']  # shape: (batch_size, seq_len)

In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]
  depths = np.arange(depth)[np.newaxis, :]/depth

  angle_rates = 1 / (10000**depths)
  angle_rads = positions * angle_rates

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [ ]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.causal_self_attention(x=x)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = x

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

  def call(self, x):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x)


    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    x  = inputs

    x = self.decoder(x)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [ ]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=vocab_size,
    target_vocab_size=vocab_size,
    dropout_rate=dropout_rate)


In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=0.1)

In [ ]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [ ]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [ ]:
inputs = input_ids[:, :-1]  # remove last token
labels = input_ids[:, 1:]   # remove first token

In [ ]:
print(inputs.shape)   # should be (batch_size, seq_len - 1)
print(labels.shape)  # should match

torch.Size([573, 511])
torch.Size([573, 511])


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.batch(32).shuffle(100).prefetch(tf.data.AUTOTUNE)

In [ ]:
transformer.fit(x=inputs, y=labels, epochs=100)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'causal_self_attention_40' (of type CausalSelfAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'decoder_layer_48' (of type DecoderLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


18/18 ━━━━━━━━━━━━━━━━━━━━ 31s 763ms/step - loss: 5.1129 - masked_accuracy: 0.2030
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 440ms/step - loss: 5.1256 - masked_accuracy: 0.2014
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 455ms/step - loss: 5.1224 - masked_accuracy: 0.2015
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 452ms/step - loss: 5.1092 - masked_accuracy: 0.2004
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 445ms/step - loss: 5.1046 - masked_accuracy: 0.2022
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 431ms/step - loss: 5.1105 - masked_accuracy: 0.2009
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 426ms/step - loss: 5.1213 - masked_accuracy: 0.1993
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 425ms/step - loss: 5.1288 - masked_accuracy: 0.1973
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 428ms/step - loss: 5.1249 - masked_accuracy: 0.2006
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 435ms/step - loss: 5.1496 - masked_accuracy: 0.1984
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 435ms/step - loss: 5

In [ ]:
output = transformer(inputs[20:21])

In [ ]:
predicted_token_ids = tf.argmax(output, axis=-1)

In [ ]:
decoded_text = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)

In [ ]:
decoded_text

"love the : the : the,,se :ssy. i permission., the ande, the, and the,. the love, world., and it the love and, and the the eyes, the -.s,, of the eyesy, the, the the the, world '.ys and the worlds and the, world, the.h,., the, and the the world, the and the,, the,, the of,h,, and the the worlds the inh,,,. and the the world,,,,,, and the lovey, and the world, and the'to the eyess, and, a lovesss,,. the. theeeee,,,,ee,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,.e,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,e,,s,,,,,,,,,,,,,,,,,,,,.eee,,,,,,,,,,,,,,,ee,,,,,,,,,,....,,,,,,,,,,,,,,,,,,,,ee,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,ee,,,,,,,,,,,,e,,,,e,,,,,,eee,,,,,,,,,,,eeee,,,,,,,,,,,,ee,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,.."